# Классификация токсичных правок

Заказчик (интернет-магазин) запускает новый сервис: теперь пользователи могут редактировать и дополнять описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Имеется набор данных с разметкой о токсичности правок. Необходимо обучить модель классифицировать комментарии на позитивные и негативные.

Условие: метрика качества *F1* должна быть не меньше 0,75. 

**План работы:**

1. Загрузить и подготовить данные.
2. Обучить разные модели с помощью <i>TF-IDF</i> и <i>BERT</i>. 
3. Выбрать лучшую модель на тестовой выборке.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## 1 Подготовка данных

Импортируем необходимые библиотеки, откроем файл с данными и изучим общую информацию о них.

In [1]:
# импорт библиотек
import nltk
import numpy as np
import pandas as pd
import spacy
import torch
import transformers

# импорт структур, модулей и функций
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords as nltk_stopwords
from pytorch_pretrained_bert import BertTokenizer, BertModel
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from tqdm import notebook
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
import re
import time

In [2]:
# константа, фиксирующая случайность
rs = 12345

In [3]:
# чтение файла с данными и сохранение в таблицу df
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/toxic_comments.csv')
    
# просмотр первых пяти строк полученной таблицы
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
# просмотр размера таблицы
df.shape

(159571, 2)

In [5]:
# просмотр количества пропусков
df.isna().sum()

text     0
toxic    0
dtype: int64

In [6]:
# просмотр количества дубликатов
df.duplicated().sum()

0

В данных 159571 правка с разметкой о токсичности, пропусков и дубликатов в столбцах нет.

Посмотрим, как соотносятся между собой положительные и отрицательные классы.

In [7]:
# определение размера сгруппированных значений
# целевого признака в исходных данных
count_class = df.groupby('toxic').size()
count_class

toxic
0    143346
1     16225
dtype: int64

Данные несбалансированы: положительного класса почти в 9 раз меньше, чем отрицательного. Учтём это при обучении моделей с помощью балансировки классов.

Подготовим правки для обучения моделей. Произведём:
- лемматизацию;
- очистку текста от лишних символов и пробелов, оставив только латиницу и одиночные пробелы.

In [8]:
# загрузка английского токенайзера
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# создание собственной функции для подготовки текста
def clearing_text(text):
  '''
  функция очистки текста: принимает на вход текст,
  возвращает текст, лемматизированный и очищенный
  от лишних символов и пробелов
  '''
  text = text.lower()
  doc = nlp(text)
  lemm_text = " ".join([token.lemma_ for token in doc])
  cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text)

  return " ".join(cleared_text.split())

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [9]:
%%time
# создание нового столбца с подготовленным текстом
df['text_lemm'] = df['text'].apply(clearing_text)
display(df.head())

,text,toxic,text_lemm
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour I be see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more I can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


CPU times: user 23min 41s, sys: 9.37 s, total: 23min 50s
Wall time: 23min 49s


### 1.1 Подготовка признаков для *TF-IDF*

Выделим новый столбец в признак, а метки о токсичности - в целевой признак. После чего произведём разбивку переменных на обучающую и тестовую выборки в соотношении 4:1. Добавим стратификацию по целевому признаку, потому что классы не сбалансированы.

In [10]:
# создание признаков и целевого признака
X = df['text_lemm']
y = df['toxic']

In [11]:
# разбивка данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=rs,
                                                    stratify=y)

display(f'Размер признаков обучающей выборки: {X_train.shape}')
display(f'Размер целевого признака обучающей выборки: {y_train.shape}')
display(f'Размер признаков тестовой выборки: {X_test.shape}')
display(f'Размер целевого признака тестовой выборки: {y_test.shape}')

'Размер признаков обучающей выборки: (127656,)'

'Размер целевого признака обучающей выборки: (127656,)'

'Размер признаков тестовой выборки: (31915,)'

'Размер целевого признака тестовой выборки: (31915,)'

### 1.2 Подготовка признаков для *BERT*

Создадим копию таблицы и сохраним в ней 5000 правок для работы *BERT*.

In [12]:
df_bert = df.copy(deep=True)
df_bert = df_bert.sample(5000).reset_index(drop=True) 
df_bert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       5000 non-null   object
 1   toxic      5000 non-null   int64 
 2   text_lemm  5000 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [13]:
# проверка дисбаланса классов в выборке для bert
count_class = df_bert.groupby('toxic').size()
count_class

toxic
0    4501
1     499
dtype: int64

Дисбаланс классов практически такой же, как в исходных данных.

Инициализуем предобученную модель *BERT* и токенизатор, токенизизируем тексты, добавим маркеры начала и конца предложения и преобразуем тексты в эмбеддинги.

In [14]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [15]:
# инициализация предобученной модели bert и токенизатора
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
                                                    transformers.DistilBertTokenizer,
                                                    'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# токенизация текста
tokenized = df_bert['text_lemm'].apply(
    lambda x: tokenizer.encode(x, truncation=True, max_length=512,
                               add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# применение padding к векторам
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# создание маски для важных токенов 
attention_mask = np.where(padded != 0, 1, 0)

display(attention_mask.shape)

(5000, 512)

In [17]:
%%time
# преобразование текста в embeddings
batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
  batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda()
  attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
 
  with torch.no_grad():
    model.cuda()
    batch_embeddings = model(batch, attention_mask=attention_mask_batch)
 
  embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 1min 32s, sys: 929 ms, total: 1min 33s
Wall time: 1min 33s


Тексты преобразованы в эмбеддинги. Теперь создадим из них признак, целевой признак и  разделим новые переменные на обучающую и тестовую выборки в соотношении 4:1. Добавим стратификацию по целевому признаку, потому что классы не сбалансированы.

In [18]:
# создание признаков и целевого признака
Xb = np.concatenate(embeddings)
yb = df_bert['toxic']

In [19]:
# разбивка данных на обучающую и тестовую выборки
Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb,
                                                    test_size=0.2,
                                                    random_state=rs,
                                                    stratify=yb)

display(f'Размер признаков обучающей выборки: {Xb_train.shape}')
display(f'Размер целевого признака обучающей выборки: {yb_train.shape}')
display(f'Размер признаков тестовой выборки: {Xb_test.shape}')
display(f'Размер целевого признака тестовой выборки: {yb_test.shape}')

'Размер признаков обучающей выборки: (4000, 768)'

'Размер целевого признака обучающей выборки: (4000,)'

'Размер признаков тестовой выборки: (1000, 768)'

'Размер целевого признака тестовой выборки: (1000,)'

### 1.3 Вывод
В данных 159571 правка с разметкой о токсичности.

Данные несбалансированы: положительного класса почти в 9 раз меньше, чем отрицательного. Учтём это при обучении моделей с помощью балансировки классов.

Произведена подготовка правок для обучения моделей:
- лемматизация;
- очистка текста от лишних символов и пробелов, оставлены только латиница и одиночные пробелы;
- для модели *BERT* создана выборка из исходных данных длиной в 500 правок, на оснвое которой созданы эмбединги.

Созданы новые переменные:
- лемматизированный и очищенный текст - признак для обучения c *TF-IDF*;
- эмбеддинги - признак для обучения с *BERT*;
- метки о токсичности - целевой признак.

Произведена разбивка переменных на обучающую и тестовую выборки в соотношении 4:1. Добавлена стратификация по целевому признаку, потому что классы не сбалансированы.

## Обучение c *TF-IDF*

Подберём лучшие гиперпараметры и обучим логистическую регрессиию, дерево решений, случайный лес, модель градиентного бустинга *LightGBMClassifier*. Для проверки адекватности моделей обучим также константную модель.

Для подбора лучших гиперпараметров используем *GridSearch*.

Используем *TF-IDF* внутри *pipeline* для того, чтобы векторизатор не обучался на валидации внутри кросс-валидации.

Для обучения и подбора напишем собственную функцию.

In [20]:
# определение списка стоп-слов
stopwords = set(nltk_stopwords.words('english'))

In [21]:
# создание функции для подбора параметров лучшей модели
def pipeline(model, X, y, params):
  '''
  функция для подбора лучших гиперпараметров и обучения модели:
  на вход принимает модель с параметрами, признаки и целевой признак;
  возвращает значение F1, обученную модель и выводит на экран
  значения F1 и подобранные гиперпараметры
  '''
  pipe = Pipeline([('vect', TfidfVectorizer(stop_words=stopwords)),
                   ('tfidf', TfidfTransformer()),
                   ('model', model)])
  param = params

  grid = GridSearchCV(pipe, param,
                      scoring='f1',
                      cv=3)
  
  grid.fit(X, y)

  display('Лучшее значение F1:', grid.best_score_)
  display('Лучшие параметры', grid.best_params_)
  
  return grid.best_score_, grid

### 2.1 Логистическая регрессия

In [22]:
%%time
# инициализация модели логистической регрессии
model_lr = LogisticRegression(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_lr = {'model__C': [0.5, 1.0],
            'model__max_iter': [100, 150],
            'model__solver': ['lbfgs', 'sag']}

f1_train_lr, grid_lr = pipeline(model_lr, X_train, y_train, param_lr)

'Лучшее значение F1:'

0.7575374066932222

'Лучшие параметры'

{'model__C': 1.0, 'model__max_iter': 100, 'model__solver': 'sag'}

CPU times: user 3min 38s, sys: 26.3 s, total: 4min 4s
Wall time: 3min 35s


На обучающей выборке получено *F1* = 0,757. Значение удовлетворяет условию (не меньше 0,75).

### 2.2 Дерево решений

In [23]:
%%time
# инициализация модели дерева решений 
model_dt = DecisionTreeClassifier(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_dt = {'model__max_depth': [35, 40],
            'model__min_samples_leaf': [2, 5]}

f1_train_dt, grid_dt = pipeline(model_dt, X_train, y_train, param_dt)

'Лучшее значение F1:'

0.6585147615140836

'Лучшие параметры'

{'model__max_depth': 40, 'model__min_samples_leaf': 2}

CPU times: user 5min 54s, sys: 825 ms, total: 5min 55s
Wall time: 5min 53s


На обучающей выборке получено *F1* = 0,659. Значение не удовлетворяет условию (не меньше 0,75).

### 2.3 Случайный лес

In [24]:
%%time
# инициализация модели случайного леса
model_rf = RandomForestClassifier(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_rf = {'model__n_estimators': [200],
            'model__max_depth': [50, 60]}

f1_train_rf, grid_rf = pipeline(model_rf, X_train, y_train, param_rf)

'Лучшее значение F1:'

0.5125696635349778

'Лучшие параметры'

{'model__max_depth': 60, 'model__n_estimators': 200}

CPU times: user 12min 3s, sys: 1.6 s, total: 12min 4s
Wall time: 12min


На обучающей выборке получено *F1* = 0,513. Значение не удовлетворяет условию (не меньше 0,75). Данная модель может показать немного лучший результат: так, при 400 деревьях и максимальной глубине, равной, 80, *F1* = 0,54. Но значение всё равно далеко от 0,75, и код выполняется очень долго.

### 2.4 *LightGBMClassifier*

In [25]:
%%time
# инициализация модели
model_lgbm = LGBMClassifier(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_lgbm = {'model__max_depth': [20],
              'model__n_estimators': [600, 700]}

f1_train_lgbm, grid_lgbm = pipeline(model_lgbm, X_train, y_train, param_lgbm)

'Лучшее значение F1:'

0.7664336937410704

'Лучшие параметры'

{'model__max_depth': 20, 'model__n_estimators': 600}

CPU times: user 33min 47s, sys: 4.61 s, total: 33min 52s
Wall time: 33min 41s


На обучающей выборке получено *F1* = 0,766. Значение удовлетворяет условию (не меньше 0,75).

### 2.5 Константная модель

In [26]:
%%time
# инициализация константной модели
model_dc = DummyClassifier(constant = 1)

# установка диапазонов гиперпараметров
param_dc = {'model__strategy': ['most_frequent', 'constant']}

f1_train_dc, grid_dc = pipeline(model_dc, X_train, y_train, param_dc)

'Лучшее значение F1:'

0.18459000522044622

'Лучшие параметры'

{'model__strategy': 'constant'}

CPU times: user 40.7 s, sys: 248 ms, total: 40.9 s
Wall time: 40.8 s


На обучающей выборке получено *F1* = 0,185 при стратегии предсказания любой правки токсичной. Значение много меньше полученных рассмотренными моделями, значит, всё рассмотренные модели адекватны.

### 2.6 Сравнение моделей и проверка качества лучшей модели
Из рассмотренных моделей требуемое значение *F1* на обучающей выборке получено логистической регрессией и *LightGBMClassifier*. При этом лучшая метрика у последней.

Константная модель показывает метрику много меньше полученных рассмотренными моделями, значит, они адекватны.

Полученные на обучающей выборке значения *F1* приведены в таблице.

In [27]:
# создание таблицы с метриками для рассмотренных моделей
result = pd.DataFrame(data=[['LogisticRegression', f1_train_lr],
                            ['DecisionTree', f1_train_dt],
                            ['RandomForest', f1_train_rf],
                            ['LightGBM', f1_train_lgbm],
                            ['Dummy', f1_train_dc]],
                      columns=['Модель', 'F1, обучающая выборка'])
display(result)

,Модель,"F1, обучающая выборка"
0,LogisticRegression,0.757537
1,DecisionTree,0.658515
2,RandomForest,0.512570
3,LightGBM,0.766434
4,Dummy,0.184590


Проверим качество лучшей модели - *LightGBMClassifier*: получим предсказания и значение *F1* на тестовой выборке.

In [28]:
# получение предсказаний лучшей модели и расчёт f1
predict_test_lgbm = grid_lgbm.predict(X_test)
f1_test_lgbm = f1_score(y_test, predict_test_lgbm)

display('F1 на тестовой выборке:', f1_test_lgbm)

'F1 на тестовой выборке:'

0.7769387162358409

### 2.7 Вывод
Подобраны лучшие гиперпараметры и обучены логистическая регрессиия, дерево решений, случайный лес, модель градиентного бустинга *LightGBMClassifier*.

При использовании *TF-IDF* из рассмотренных моделей требуемое значение *F1* (не меньше 0,75) на обучающей выборке получено логистической регрессией и *LightGBMClassifier*. При этом лучшая метрика у последней.

Константная модель даёт наихудшую метрику, значит, обученные модели адекватны.

На тестовой выборке *LightGBMClassifier* получено значение F1 = 0,777, что также удовлетворяет условию.

## 3 Обучение с *BERT*

Подберём лучшие гиперпараметры и обучим логистическую регрессиию, дерево решений, случайный лес, модель градиентного бустинга *LightGBMClassifier*. Для проверки адекватности моделей обучим также константную модель.

Для подбора лучших гиперпараметров используем *GridSearch*.

Для обучения и подбора напишем собственную функцию.

In [29]:
# создание функции для подбора параметров лучшей модели
def pipeline_b(model, X, y, params):
  '''
  функция для подбора лучших гиперпараметров и обучения модели:
  на вход принимает модель с параметрами, возвращает значения
  F1 на обучающей и тестовой выборках, лучшую модель
  и выводит на экран значения F1 и подобранные гиперпараметры
  '''
  pipe = Pipeline([('model', model)])
  param = params

  grid = GridSearchCV(pipe, param,
                      scoring='f1',
                      cv=3)
  
  grid.fit(X, y)

  display('Лучшее значение F1:', grid.best_score_)
  display('Лучшие параметры', grid.best_params_)
  
  return grid.best_score_, grid

### Логистическая регрессия

In [30]:
%%time
# инициализация модели логистической регрессии
model_lr_b = LogisticRegression(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_lr_b = {'model__C': [0.5, 1.0],
            'model__max_iter': [400, 500],
            'model__solver': ['lbfgs', 'sag']}

f1_train_lr_b, grid_lr_b = pipeline_b(model_lr_b, Xb_train, yb_train, param_lr_b)

'Лучшее значение F1:'

0.6674523648932319

'Лучшие параметры'

{'model__C': 1.0, 'model__max_iter': 400, 'model__solver': 'sag'}

CPU times: user 2min 3s, sys: 5.44 s, total: 2min 9s
Wall time: 1min 57s


На обучающей выборке получено *F1* = 0,667. Значение не удовлетворяет условию (не меньше 0,75).

### Дерево решений

In [31]:
%%time
# инициализация модели дерева решений 
model_dt_b = DecisionTreeClassifier(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_dt_b = {'model__max_depth': range(10, 20),
            'model__min_samples_leaf': [2, 5]}

f1_train_dt_b, grid_dt_b = pipeline_b(model_dt_b, Xb_train, yb_train, param_dt_b)

'Лучшее значение F1:'

0.49957140074030776

'Лучшие параметры'

{'model__max_depth': 11, 'model__min_samples_leaf': 2}

CPU times: user 1min 40s, sys: 142 ms, total: 1min 40s
Wall time: 1min 40s


На обучающей выборке получено *F1* = 0,5. Значение не удовлетворяет условию (не меньше 0,75).

### Случайный лес

In [32]:
%%time
# инициализация модели случайного леса
model_rf_b = RandomForestClassifier(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_rf_b = {'model__n_estimators': range(35, 46, 5),
            'model__max_depth': range(3, 6)}

f1_train_rf_b, grid_rf_b = pipeline_b(model_rf_b, Xb_train, yb_train, param_rf_b)

'Лучшее значение F1:'

0.6619736575669907

'Лучшие параметры'

{'model__max_depth': 5, 'model__n_estimators': 45}

CPU times: user 28.6 s, sys: 72.7 ms, total: 28.6 s
Wall time: 28.7 s


На обучающей выборке получено *F1* = 0,66. Значение не удовлетворяет условию (не меньше 0,75).

### *LightGBMClassifier*

In [33]:
%%time
# инициализация модели
model_lgbm_b = LGBMClassifier(class_weight='balanced', random_state=rs)

# установка диапазонов гиперпараметров
param_lgbm_b = {'model__max_depth': range(1, 3),
              'model__n_estimators': [1300, 1350]}

f1_train_lgbm_b, grid_lgbm_b = pipeline_b(model_lgbm_b, Xb_train, yb_train, param_lgbm_b)

'Лучшее значение F1:'

0.6659859430475678

'Лучшие параметры'

{'model__max_depth': 1, 'model__n_estimators': 1300}

CPU times: user 3min 56s, sys: 634 ms, total: 3min 56s
Wall time: 3min 59s


На обучающей выборке получено *F1* = 0,666. Значение не удовлетворяет условию (не меньше 0,75).

### Константная модель

In [34]:
%%time
# инициализация фиктивной модели
model_dc_b = DummyClassifier(constant = 1)

# установка диапазонов гиперпараметров
param_dc_b = {'model__strategy': ['most_frequent', 'constant']}

f1_train_dc_b, grid_dc_b = pipeline_b(model_dc_b, Xb_train, yb_train, param_dc_b)

'Лучшее значение F1:'

0.18140488348642084

'Лучшие параметры'

{'model__strategy': 'constant'}

CPU times: user 58.6 ms, sys: 3 ms, total: 61.6 ms
Wall time: 82.2 ms


На обучающей выборке получено *F1* = 0,181 при стратегии предсказания любой правки токсичной. Значение много меньше полученных рассмотренными моделями, значит, всё рассмотренные модели адекватны.

### Вывод

При использовании *BERT* c небольшой выборкой (5000 объектов) ни одна из рассмотренных моделей не показала требуемое качество. Наибольшее значение *F1* на обучающей выборке у логистической регрессии и *LightBGMClassifier*.

In [35]:
# создание таблицы с показателями рассмотренных моделей
result_b = pd.DataFrame(data=[['LogisticRegression', f1_train_lr_b],
                            ['DecisionTree', f1_train_dt_b],
                            ['RandomForest', f1_train_rf_b],
                            ['LightGBM', f1_train_lgbm_b],
                            ['Dummy', f1_train_dc_b]],
                      columns=['Модель', 'F1, обучающая выборка'])
display(result_b)

,Модель,"F1, обучающая выборка"
0,LogisticRegression,0.667452
1,DecisionTree,0.499571
2,RandomForest,0.661974
3,LightGBM,0.665986
4,Dummy,0.181405


## Общий вывод
1. В данных 159571 правка с разметкой о токсичности.

- Данные несбалансированы: положительного класса почти в 9 раз меньше, чем отрицательного. Учтём это при обучении моделей с помощью балансировки классов.

2. Произведена подготовка правок для обучения моделей:
- лемматизация;
- очистка текста от лишних символов и пробелов, оставлены только латиница и одиночные пробелы;
- для модели *BERT* создана выборка из исходных данных длиной в 5000 правок, на оснвое которой созданы эмбединги.

3. Созданы новые переменные:
- лемматизированный и очищенный текст - признак для обучения с *TF-IDF*;
- эмбеддинги - признак для обучения с *BERT*;
- метки о токсичности - целевой признак.

4. Произведена разбивка переменных на обучающую и тестовую выборки в соотношении 4:1. Добавлена стратификация по целевому признаку, потому что классы не сбалансированы.

5. Подобраны лучшие гиперпараметры и обучены логистическая регрессиия, дерево решений, случайный лес, модель градиентного бустинга *LightGBMClassifier*.

6. Из рассмотренных моделей требуемое значение *F1* (не меньше 0,75) на обучающей выборке получено при использовании *TF-IDF* логистической регрессией и *LightGBMClassifier*. При этом лучшая метрика у последней.

- При использовании *BERT* c небольшой выборкой (5000 объектов) ни одна из рассмотренных моделей не показала требуемое качество.

- Константная модель показывает метрику много меньше полученных рассмотренными моделями, значит, они адекватны.

- На тестовой выборке у *LightGBMClassifier* *F1* = 0,777. Значит, для анализа токсичности правок можно рекомендовать эту модель с 600 деревьями и с глубиной дерева, равной 20.